In [3]:
library(ggplot2)
library(dplyr)
library(data.table)
library(tidyr)

In [4]:
df <- fread(snakemake@input[[1]]) %>%
    select(Measure = V1, Lineage = V2, Value = V3)


In [5]:
#merge chronumental
dfs <- list()
ct = 0 
for (ch in snakemake@input$chronumental) {
    ct = ct + 1
    dfs[[ct]] <- fread(ch)
}
df_chr <- bind_rows(dfs) %>%
    select(Lineage = LINEAGE,CHR_startER = START_MUTATION_RATE, CHR_ER = MUTATION_RATE ) %>%
    pivot_longer(cols = c("CHR_startER","CHR_ER"), values_to = "Value", names_to = "Measure")
df <- bind_rows(df,df_chr) %>%
    rowwise() %>%
    filter(Measure %in% c("CHR_startER","CHR_ER","rate"))

In [6]:

df4rank <- df %>%
    group_by(Lineage) %>%
    summarise(M=mean(as.numeric(Value))) %>%
    arrange(-M)
df$Lineage <- factor(df$Lineage, levels = df4rank$Lineage)

In [7]:
p <- ggplot(data = df, aes(x = Lineage, y = Value, color = Lineage)) + geom_point() + facet_grid(Measure ~ ., scales = "free")
p

In [8]:
fwrite(x = df, snakemake@output[[1]])

In [10]:
fwrite(x = df, "iq1_unfiltered_seqs_chrom_on_timetree.csv")